In [248]:
pip install azure-monitor-query

Note: you may need to restart the kernel to use updated packages.


In [2]:
from azure.identity import DefaultAzureCredential
from azure.monitor.query import LogsQueryClient, MetricsQueryClient, MetricsClient

In [3]:
import os
import ast
import pandas as pd
from datetime import datetime, timezone
from azure.monitor.query import LogsQueryClient, LogsQueryStatus
from azure.identity import DefaultAzureCredential
from azure.core.exceptions import HttpResponseError
import numpy as np

In [4]:
credential = DefaultAzureCredential()
logs_query_client = LogsQueryClient(credential)


In [5]:
client = LogsQueryClient(credential)


start_time=datetime(2024, 4, 2, tzinfo=timezone.utc)
end_time=datetime(2024, 5, 4, tzinfo=timezone.utc)

query_conversation_data = ''' AppTraces | project TimeGenerated, Message, Properties | where Message == "conversation_data" '''
 
query_llm_data = '''  AppTraces | project TimeGenerated, Message, Properties | where Message == "llm_data" '''


In [11]:



start_time='2024/04/01'
end_time='2024/05/05'

#Converting the start_time and end_time to datetime with timezone
start_time=datetime.strptime(start_time, '%Y/%m/%d').replace(tzinfo=timezone.utc)
end_time=datetime.strptime(end_time, '%Y/%m/%d').replace(tzinfo=timezone.utc)

query_conversation_data = ''' AppTraces | project TimeGenerated, Message, Properties | where Message == "conversation_data" '''
 
query_llm_data = '''  AppTraces | project TimeGenerated, Message, Properties | where Message == "llm_data" '''

In [12]:
try:
    response = client.query_workspace(
        workspace_id="24bbb4b3-a8e3-4a98-9c0d-2a48494c5e35",
        query=query_conversation_data,
        timespan=(start_time, end_time)
        )
    if response.status == LogsQueryStatus.PARTIAL:
        error = response.partial_error
        data = response.partial_data
        print(error)
    elif response.status == LogsQueryStatus.SUCCESS:
        data = response.tables
    for table in data:
        df_conversation = pd.DataFrame(data=table.rows, columns=table.columns)
        print(df_conversation)
except HttpResponseError as err:
    print("something fatal happened")
    print(err)

                       TimeGenerated            Message  \
0   2024-05-02 05:41:27.042375+00:00  conversation_data   
1   2024-05-02 05:41:45.188098+00:00  conversation_data   
2   2024-05-01 12:02:21.396239+00:00  conversation_data   
3   2024-05-01 12:02:25.091355+00:00  conversation_data   
4   2024-05-01 12:02:29.274942+00:00  conversation_data   
..                               ...                ...   
869 2024-05-01 13:15:20.151047+00:00  conversation_data   
870 2024-05-01 13:15:26.128892+00:00  conversation_data   
871 2024-05-01 13:15:30.873785+00:00  conversation_data   
872 2024-05-01 13:15:36.319355+00:00  conversation_data   
873 2024-05-01 13:15:41.144701+00:00  conversation_data   

                                            Properties  
0    {"code.filepath":"E:\\code\\llm-eval-grader\\src\\...  
1    {"code.filepath":"E:\\code\\llm-eval-grader\\src\\...  
2    {"code.filepath":"E:\\code\\Evaluations\\Sampl...  
3    {"code.filepath":"E:\\code\\Evaluations\\Sampl... 

In [254]:
try:
    response = client.query_workspace(
        workspace_id="24bbb4b3-a8e3-4a98-9c0d-2a48494c5e35",
        query=query_llm_data,
        timespan=(start_time, end_time)
        )
    if response.status == LogsQueryStatus.PARTIAL:
        error = response.partial_error
        data = response.partial_data
        print(error)
    elif response.status == LogsQueryStatus.SUCCESS:
        data = response.tables
    for table in data:
        df_llm = pd.DataFrame(data=table.rows, columns=table.columns)
        print(df_llm)
except HttpResponseError as err:
    print("something fatal happened")
    print(err)

                        TimeGenerated   Message  \
0    2024-05-02 05:41:27.027795+00:00  llm_data   
1    2024-05-02 05:41:27.027795+00:00  llm_data   
2    2024-05-02 05:41:45.183121+00:00  llm_data   
3    2024-05-02 05:41:45.184253+00:00  llm_data   
4    2024-05-01 12:02:21.390342+00:00  llm_data   
...                               ...       ...   
1743 2024-05-01 13:15:30.871311+00:00  llm_data   
1744 2024-05-01 13:15:36.312971+00:00  llm_data   
1745 2024-05-01 13:15:36.315937+00:00  llm_data   
1746 2024-05-01 13:15:41.141038+00:00  llm_data   
1747 2024-05-01 13:15:41.142272+00:00  llm_data   

                                             Properties  
0     {"code.filepath":"E:\\code\\llm-eval-grader\\src\\...  
1     {"code.filepath":"E:\\code\\llm-eval-grader\\src\\...  
2     {"code.filepath":"E:\\code\\llm-eval-grader\\src\\...  
3     {"code.filepath":"E:\\code\\llm-eval-grader\\src\\...  
4     {"code.filepath":"E:\\code\\Evaluations\\Sampl...  
...                    

In [255]:
df_llm.shape

(1748, 3)

In [256]:
df_conversation_mapped = pd.DataFrame(columns=["conversation_id", "turn_id", "query", "response", "timestamp"])
for i in range(df_conversation.shape[0]):
    df_conversation_mapped.loc[i] = [ast.literal_eval(df_conversation["Properties"].iloc[i])["conversation_id"], ast.literal_eval(df_conversation["Properties"].iloc[i])["turn_id"], ast.literal_eval(df_conversation["Properties"].iloc[i])["query"], ast.literal_eval(df_conversation["Properties"].iloc[i])["response"], df_conversation["TimeGenerated"].iloc[i]]

In [257]:
df_conversation_mapped.head()

,conversation_id,turn_id,query,response,timestamp
0,22851dd3-1db6-4c76-9ee1-14cb73730b4f,9df607a1-6e27-47ce-a173-f50c819a3fde,Suggest me some good TV,"Sure, I'd be happy to help you find a good TV....",2024-05-02 05:41:27.042375+00:00
1,22851dd3-1db6-4c76-9ee1-14cb73730b4f,9c4dc198-796a-46fb-8de9-37830ad8482a,under 10K,"Sure, there are several good options available...",2024-05-02 05:41:45.188098+00:00
2,0f30a5e1-7e40-4595-bdf4-87ad85b4fb26,730c1f83-1306-45e8-9ce5-346545a7c7f4,What's the best aisle to find organic produce?,"Typically, organic produce can be found in a s...",2024-05-01 12:02:21.396239+00:00
3,89cd78c4-1004-4ee6-905b-f9cb7fb798ff,95e28832-8d11-4b57-8596-dd15d9c94e17,Where can I find HDMI cables?,You can find HDMI cables in most electronics s...,2024-05-01 12:02:25.091355+00:00
4,5f60fc63-c8a9-47a3-9bcd-3a5c9d224ee6,6dfea0a1-37cf-4d28-93e7-fd7a89299676,Where can I find HDMI cables?,You can find HDMI cables at most electronic st...,2024-05-01 12:02:29.274942+00:00


In [287]:

df_llm_mapped = pd.DataFrame(columns=["context","response","conversation_id", "turn_id", "query","intent", "model","timestamp"])
 

In [ ]:
#Remove the rows where llm_response is null or empty in the df_llm
# if ast.literal_eval(df_llm["Properties"].iloc[0])("llm_response") is null then drop those rowa from df_llm


In [288]:
for i in range(df_llm.shape[0]):
    if "llm_response" not in ast.literal_eval(df_llm["Properties"].iloc[i]):
        continue 
    else:
        if "context" in ast.literal_eval(df_llm["Properties"].iloc[i]):
            df_llm_mapped.loc[i] = [ast.literal_eval(df_llm["Properties"].iloc[i])["context"],ast.literal_eval(df_llm["Properties"].iloc[i])["llm_response"],ast.literal_eval(df_llm["Properties"].iloc[i])["conversation_id"], ast.literal_eval(df_llm["Properties"].iloc[i])["turn_id"], ast.literal_eval(df_llm["Properties"].iloc[i])["query"], ast.literal_eval(df_llm["Properties"].iloc[i])["intent"], ast.literal_eval(df_llm["Properties"].iloc[i])["model"], df_llm["TimeGenerated"].iloc[i]]
        else:
            df_llm_mapped.loc[i] = ["",ast.literal_eval(df_llm["Properties"].iloc[i])["llm_response"],ast.literal_eval(df_llm["Properties"].iloc[i])["conversation_id"], ast.literal_eval(df_llm["Properties"].iloc[i])["turn_id"], ast.literal_eval(df_llm["Properties"].iloc[i])["query"], ast.literal_eval(df_llm["Properties"].iloc[i])["intent"], ast.literal_eval(df_llm["Properties"].iloc[i])["model"], df_llm["TimeGenerated"].iloc[i]]

In [289]:
df_llm_mapped.head()

,context,response,conversation_id,turn_id,query,intent,model,timestamp
0,,"{\n ""id"": ""chatcmpl-9KJU6EnHm7P4YPJ2A6tUNL8Z2...",22851dd3-1db6-4c76-9ee1-14cb73730b4f,9df607a1-6e27-47ce-a173-f50c819a3fde,Suggest me some good TV,electronics,electronics_model,2024-05-02 05:41:27.027795+00:00
1,,"{\n ""id"": ""chatcmpl-9KJU65RhbUbO3senbJTwq7I4i...",22851dd3-1db6-4c76-9ee1-14cb73730b4f,9df607a1-6e27-47ce-a173-f50c819a3fde,Suggest me some good TV,electronics,intent_detection_model,2024-05-02 05:41:27.027795+00:00
2,"\nUser:Suggest me some good TV\nBot:Sure, I'd ...","{\n ""id"": ""chatcmpl-9KJUMRY4OGNJ2Y6HEh5LkgnLb...",22851dd3-1db6-4c76-9ee1-14cb73730b4f,9c4dc198-796a-46fb-8de9-37830ad8482a,under 10K,electronics,intent_detection_model,2024-05-02 05:41:45.183121+00:00
3,"\nUser:Suggest me some good TV\nBot:Sure, I'd ...","{\n ""id"": ""chatcmpl-9KJUM8HgdCCEtQrkbAXbjGMbd...",22851dd3-1db6-4c76-9ee1-14cb73730b4f,9c4dc198-796a-46fb-8de9-37830ad8482a,under 10K,electronics,electronics_model,2024-05-02 05:41:45.184253+00:00
92,,"{\n ""id"": ""chatcmpl-9K33QsrRbxxC4SnmQsKUdYKyA...",5ea70872-ada2-42a9-b9cd-cd570f3e5c47,59d57d08-36d5-492f-8e52-d7152fc63ea0,What can you do for me,general,intent_detection_model,2024-05-01 12:08:49.158175+00:00


In [290]:
df_llm_mapped["response"] = df_llm_mapped["response"].apply(lambda x: json.loads(x)["choices"][0]["message"]["content"] if x != "" else x)

In [291]:
df_llm_mapped.shape

(1654, 8)